In [1]:
import os
import re
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
niftis = os.listdir('/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA')
tcia_pid = set([int(x) for x in niftis])

In [8]:
pids = []
study_yrs = []
outputnames = []

for pid in tcia_pid:
    if os.path.exists(f'/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/{pid}/'):
        original_dir = f'/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/{pid}/'
        files = os.listdir(original_dir)
        end_with_0 = [file for file in files if file.endswith('0_0000.nii.gz')]
        end_with_1 = [file for file in files if file.endswith('1_0000.nii.gz')]
        end_with_2 = [file for file in files if file.endswith('2_0000.nii.gz')]

        recon_0 = [x.split("$")[3] for x in end_with_0]
        recon_1 = [x.split("$")[3] for x in end_with_1]
        recon_2 = [x.split("$")[3] for x in end_with_2]

        slices_0 = [float(x.split("$")[4]) for x in end_with_0]
        slices_1 = [float(x.split("$")[4]) for x in end_with_1]
        slices_2 = [float(x.split("$")[4]) for x in end_with_2]

        packages = [(end_with_0, recon_0, slices_0), (end_with_1, recon_1, slices_1), (end_with_2, recon_2, slices_2)]


        for lis_idx, tuple_package in enumerate(packages):
            end_names, recon_names, slice_thickness = tuple_package

            alpha_num = [(idx, i.upper()) for idx, i in enumerate(recon_names) if bool(re.search(r'\d', i)) and bool(re.search(r'[a-zA-Z]', i))]

            single_alpha = [(idx, i.upper()) for idx, i in enumerate(recon_names) if len(i)==1 and bool(re.search(r'[a-zA-Z]', i))]

            alpha = [(idx, i.upper()) for idx, i in enumerate(recon_names) if bool(re.search(r'[a-zA-Z]', i)) and not bool(re.search(r'\d', i))]

            if alpha_num:

                best_index = max(alpha_num, key=lambda x: x[1])[0]

                if len([i for i in alpha_num if i[1]==alpha_num[best_index][1]])>1:
                    same_filter_thickness_index = [i[0] for i in alpha_num if i[1]==alpha_num[best_index][1]]
                    same_filter_thickness = [(i, slice_thickness[i]) for i in same_filter_thickness_index]
                    best_index = min(same_filter_thickness, key=lambda x: x[1])[0]
                    
                output = end_names[best_index]
                
                pids.append(pid)
                study_yrs.append(lis_idx)
                outputnames.append(original_dir+output)

            elif single_alpha:
                best_index = max(single_alpha, key=lambda x: x[1])[0]

                if len([i for i in single_alpha if i[1]==single_alpha[best_index][1]])>1:
                    same_filter_thickness_index = [i[0] for i in single_alpha if i[1]==single_alpha[best_index][1]]
                    same_filter_thickness = [(i, slice_thickness[i]) for i in same_filter_thickness_index]
                    best_index = min(same_filter_thickness, key=lambda x: x[1])[0]

                output = end_names[best_index]

                pids.append(pid)
                study_yrs.append(lis_idx)
                outputnames.append(original_dir+output)

            elif alpha:
                printed = False
                for i in alpha:
                    if i[1]=='BONE':
                        printed = True
                        
                        if len([i for i in alpha if i[1]==i[1]])>1:
                            same_filter_thickness_index = [i[0] for i in alpha if i[1]==i[1]]
                            same_filter_thickness = [(i, slice_thickness[i]) for i in same_filter_thickness_index]
                            best_index = min(same_filter_thickness, key=lambda x: x[1])[0]

                        output = end_names[best_index]
                        pids.append(pid)
                        study_yrs.append(lis_idx)
                        outputnames.append(original_dir+output)
                        break

                if not printed:
                    slice_filter_thickness = [(idx, i) for idx, i in enumerate(slice_thickness)]
                    best_index = min(slice_filter_thickness, key=lambda x: x[1])[0]
                    output = end_names[best_index]
                    pids.append(pid)
                    study_yrs.append(lis_idx)
                    outputnames.append(original_dir+output)

    else:
        print(f'pid {pid} folder does not exist')

In [9]:
df = pd.DataFrame({'pid': pids, 'study_yr': study_yrs, 'outputname': outputnames})

In [10]:
# show all portion of the column, not just the part of it
df.head(10)

,pid,study_yr,outputname
0,101888,0,/local_ssd/practical_wise24/lung_cancer/images...
1,101888,1,/local_ssd/practical_wise24/lung_cancer/images...
2,101888,2,/local_ssd/practical_wise24/lung_cancer/images...
3,102402,0,/local_ssd/practical_wise24/lung_cancer/images...
4,102402,1,/local_ssd/practical_wise24/lung_cancer/images...
5,102402,2,/local_ssd/practical_wise24/lung_cancer/images...
6,100364,0,/local_ssd/practical_wise24/lung_cancer/images...
7,100364,1,/local_ssd/practical_wise24/lung_cancer/images...
8,100364,2,/local_ssd/practical_wise24/lung_cancer/images...
9,100879,0,/local_ssd/practical_wise24/lung_cancer/images...


In [11]:
df.to_csv('/local_ssd/practical_wise24/lung_cancer/prasanga/best_niftis.csv', index=False)

In [29]:
df.loc[df['pid']==108070]['outputname'].values

array(['/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/108070/108070$1.2.840.113654.2.55.120620084894928629878181449726430809468$1.2.840.113654.2.55.58504763685802334453431215899792255893$B50f$2.0$0_0000.nii.gz',
       '/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/108070/108070$1.2.840.113654.2.55.288565662367305584881237091580402903293$1.2.840.113654.2.55.168657851573393895597246164733468105726$B50f$2.0$1_0000.nii.gz',
       '/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/108070/108070$1.2.840.113654.2.55.194452679454675697720769209915726176917$1.2.840.113654.2.55.123732983349604987122846405328611093929$B50f$2.0$2_0000.nii.gz'],
      dtype=object)

In [30]:
df.loc[df['pid']==100019]['outputname'].values

array(['/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/100019/100019$1.2.840.113654.2.55.326251216458134568337847627223356709293$1.2.840.113654.2.55.195698197774937670503090134542347726476$D$3.2$0_0000.nii.gz',
       '/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/100019/100019$1.2.840.113654.2.55.304084032039431477396810497289904611246$1.2.840.113654.2.55.318103937194054943729360193795512668859$D$3.2$1_0000.nii.gz',
       '/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/100019/100019$1.2.840.113654.2.55.49952732435896735357359126866868156198$1.2.840.113654.2.55.172003219410512272495744434039221052323$D$3.2$2_0000.nii.gz'],
      dtype=object)

In [34]:
orig_dir = "/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/"
dir_lis = os.listdir(orig_dir)

In [42]:
pid_outputname = [(pid, outputname) for pid, outputname in zip(df['pid'], df['outputname'])]

In [41]:
import shutil

for pid, outputname in pid_outputname:
    if not os.path.exists(outputname):
        print(pid)
    
    else:
        dest_folder = f'/local_ssd/practical_wise24/lung_cancer/prasanga/best_niftis/{pid}/'

        if not os.path.exists(dest_folder):
            os.mkdir(dest_folder)
        
        shutil.copy(outputname, dest_folder+outputname.split('/')[-1])
        print(f'copied {outputname} to {dest_folder+outputname.split("/")[-1]}')

copied /local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/101888/101888$1.2.840.113654.2.55.309118181960568788630222690502725910665$1.2.840.113654.2.55.306800037222377078963911978039171108965$B50f$2.0$0_0000.nii.gz to /local_ssd/practical_wise24/lung_cancer/prasanga/best_niftis/101888/101888$1.2.840.113654.2.55.309118181960568788630222690502725910665$1.2.840.113654.2.55.306800037222377078963911978039171108965$B50f$2.0$0_0000.nii.gz
copied /local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/101888/101888$1.2.840.113654.2.55.258077601901704319745384755704563141257$1.2.840.113654.2.55.173530340011141110505849537426879615649$B50f$2.0$1_0000.nii.gz to /local_ssd/practical_wise24/lung_cancer/prasanga/best_niftis/101888/101888$1.2.840.113654.2.55.258077601901704319745384755704563141257$1.2.840.113654.2.55.173530340011141110505849537426879615649$B50f$2.0$1_0000.nii.gz
copied /local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA/101888/101888$1.2.8

In [1]:
import shutil
import os

source_folder = '/local_ssd/practical_wise24/lung_cancer/prasanga/best_niftis/'

dest_folder = '/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/data_dir/Task016_Luna/raw_splitted/imagesTs/'

for folder in os.listdir(source_folder):
    pid = folder.split('/')[-1]
    for file in os.listdir(source_folder+folder):
        shutil.copy(source_folder+folder+'/'+file, dest_folder+file)


In [12]:
len(os.listdir("/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/data_dir/Task016_Luna/preprocessed/D3V001_3d/imagesTs"))

1728

In [4]:
len(os.listdir("/local_ssd/practical_wise24/lung_cancer/images/converted_nifti_TCIA"))

306

In [13]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
arr = np.load('/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/data_dir/Task016_Luna/preprocessed/D3V001_3d/imagesTs/100002$1.2.840.113654.2.55.68425808326883186792123057288612355322$1.2.840.113654.2.55.257926562693607663865369179341285235858$STANDARD$2.5$0.npz')


In [15]:
arr

NpzFile '/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/data_dir/Task016_Luna/preprocessed/D3V001_3d/imagesTs/100002$1.2.840.113654.2.55.68425808326883186792123057288612355322$1.2.840.113654.2.55.257926562693607663865369179341285235858$STANDARD$2.5$0.npz' with keys: data

In [17]:
arr['data'].shape

(1, 252, 512, 512)

In [18]:
arr.keys()

KeysView(NpzFile '/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/data_dir/Task016_Luna/preprocessed/D3V001_3d/imagesTs/100002$1.2.840.113654.2.55.68425808326883186792123057288612355322$1.2.840.113654.2.55.257926562693607663865369179341285235858$STANDARD$2.5$0.npz' with keys: data)

In [21]:
import pickle

with open('/local_ssd/practical_wise24/lung_cancer/prasanga/nnDetection/models_dir/Task016_Luna/Luna16_nodule_detection_epochs_50_default_settings_rodeo_off_deterministic_true/fold2/test_predictions/100002$1.2.840.113654.2.55.68425808326883186792123057288612355322$1.2.840.113654.2.55.257926562693607663865369179341285235858$STANDARD$2.5$0_boxes.pkl', 'rb') as f:
    data = pickle.load(f)

In [22]:
data

{'pred_boxes': array([[109.41048  , 390.80774  , 112.922424 , 402.8783   , 195.7721   ,
         207.94363  ],
        [ 33.45826  , 348.32278  ,  39.77393  , 372.9279   , 154.5218   ,
         178.34753  ],
        [ 15.8395195, 333.05356  ,  19.250742 , 344.0003   , 375.7878   ,
         385.89075  ],
        [ 29.270668 , 197.84695  ,  31.977297 , 205.52678  , 196.0374   ,
         203.87148  ],
        [ 72.99031  , 258.82428  ,  75.71162  , 267.01697  , 369.1934   ,
         377.54456  ],
        [ 33.041767 , 353.13943  ,  35.899178 , 364.0139   , 224.39117  ,
         235.04735  ],
        [ 47.98679  , 159.99556  ,  50.726295 , 168.31693  , 376.       ,
         381.8726   ],
        [ 79.44943  , 211.12125  ,  82.14261  , 215.8958   , 136.       ,
         143.09576  ],
        [118.154465 , 225.96507  , 120.88222  , 234.51584  , 340.3335   ,
         348.7578   ],
        [ 55.351154 , 282.42703  ,  58.555763 , 291.72357  ,  69.8895   ,
          79.0451   ],
        [ 54.160